<a href="https://colab.research.google.com/github/ttogle918/news_by_kobert/blob/master/Spacing/Bert_Spacing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 띄어쓰기 

  1. 패키지 설치 및 선언, GPU 사용 설정
  2. 학습 데이터 준비
    - Dataset : train, validation할 데이터
    - Testset : 예측할 데이터
  3. 데이터 전처리
    - KoBertTokenizer
    - CorpusDataset
    - Preprocessor
    - Config
  4. 모델
  5. 학습
    - checkpoint를 통해 모델 저장
    - 학습 이어서하기
  6. 실제 데이터로 예측
    - 모델 불러오기 및 재사용
    -




출처 : 
* [BERT를 이용한 한국어 띄어쓰기 모델 만들기 - 01. 데이터 준비](https://bhchoi.github.io/post/nlp/dev/bert_korean_spacing_01/)
* [BERT를 이용한 한국어 띄어쓰기 모델 만들기 - 02. 데이터 전처리](https://bhchoi.github.io/post/nlp/dev/bert_korean_spacing_02/)
* [BERT를 이용한 한국어 띄어쓰기 모델 만들기 - 03. 모델](https://bhchoi.github.io/post/nlp/dev/bert_korean_spacing_03/)
* [BERT를 이용한 한국어 띄어쓰기 모델 만들기 - 04. 학습](https://bhchoi.github.io/post/nlp/dev/bert_korean_spacing_04/)
* [Docs : pytorch lightning 사용법](https://pytorch-lightning.readthedocs.io/en/latest/common/lightning_module.html)
* [Docs : pytorch lightning trainer 메소드 종류](https://pytorch-lightning.readthedocs.io/en/stable/common/trainer.html)

``` python
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git
```


+ '.'으로 끝나지 않는 문장(headline, subheadline 포함) 100가지를 PyKoSpacing으로 살펴본 결과 모두 정상 문장이었다.
+ 하지만 PyKoSpacing로 띄어쓰기 검증을 했을 때 잘못된 띄어쓰기를 하는 것을 볼 수 있었다.
+ PyKoSpacing는 ~다.로 끝나는 문장에는 띄어쓰기 결과가 거의 100% 정확도를 보이지만, 제목, 부제목과 같이 요약된 문장에는 좋지 못한 성능(60%)를 보인다.

# 패키지 설치 및 선언, GPU 사용 설정

In [ ]:
!pip install sentencepiece transformers torch

In [ ]:
!pip install seqeval
!pip install torchtext pytorch-lightning
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [1]:
import pytorch_lightning as pl
pl.__version__

'1.5.10'

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [3]:
from transformers import BertConfig, BertModel, AdamW

In [4]:
from tqdm import tqdm, tqdm_notebook
from typing import Callable, List, Tuple
from seqeval.metrics import f1_score, accuracy_score

In [5]:
import torch
# gpu 연산이 가능하면 'cuda:0', 아니면 'cpu' 출력
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device, torch.cuda.device_count()

(device(type='cuda', index=0), 1)

# 1. 학습 데이터 준비

100,000 lines 사용 ( headline, subheadline, content )

In [6]:
from google.colab import drive

drive.mount('/content/drive')
data_path = '/content/drive/My Drive/Colab Notebooks/NextLab/news_class9x1400'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def remove_text(con) :
  deleted = []
  for i, v in enumerate(con) :
    if len(v) < 8 :
      deleted.append(i)
    elif v[0] in ['o','『', '(', '┌','│', '└', 'ㄴ', '┌','├','◎', '[', '■', 'ㄱ', '-', '.', '<'] : 
      deleted.append(i)
    elif v[-1] in ['쪽', '-', '>', ']'] :
      deleted.append(i)
    elif v[1] in [')'] :
      deleted.append(i)
    elif v[0:2] in ['vs', '만,', '해!'] :
      deleted.append(i)
  for v in reversed(deleted) :
    del con[v]
  return con

In [ ]:
def text_splicing(con, max_len) :
  # if len(con) == 0 :
  #   return con
  before_len, before_idx, i = len(con[0]), 0, 1
  
  while True:
    if i >= len(con) :
      break
    before_len += len(con[i])
    if before_len < max_len :
      con[before_idx] += ' '+ con[i]
      del con[i]    ## i++하면 안됌!
    else :
      before_idx = i
      i += 1
      before_len= len(con[before_idx])
  return con

In [7]:
# max_len : bert에 넣을 문장의 최대 길이 ( 이상이면 잘리고, 적으면 padding이 들어간다.)
max_len = 128

### dataset

In [ ]:
import os
content = []
i = 0
for (path, dir, files) in os.walk(data_path):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.txt':
            with open("%s/%s" % (path, filename), encoding="utf-8") as f:
              label = path[path.rindex('/')+1:]
              text = f.read()
              text = text.split('\n')
              temp = []
              for t in text :
                temp.append(t)
              temp = remove_text(temp)
              temp = text_splicing(temp, max_len)
              content.extend(temp)
              
len(content)

286502

In [ ]:
from sklearn.model_selection import train_test_split

# Split Train and Validation data
train, val = train_test_split(content, test_size=0.2, random_state=0, shuffle=True)
print(len(train), len(val))

229201 57301


### testset

In [ ]:
import requests

from bs4 import BeautifulSoup
from bs4 import CData
import os

In [8]:
testdata_path = '/content/drive/Othercomputers/내 노트북/신문원문PDF'

In [ ]:
# test, 128자를 넘으면 추가 slicing 해야하지만, 넘지 않았다.
te = '윤 당선인은 이날 현판식 직후 인수위 첫 전체회의를 주재하고 “국정과제 수립에 있어 국가의 안보, 국민의 민생에 한 치의 빈틈이 없어야 한다”며 이같이 강조했다. 이어 “국정과제 우선순위 설정은 궁극적으로 국민통합을 위한 것”이라며 “국민께서 어느 지역에 사느냐와 관계없이 공정한 기회를 보장받아야 한다”고 했다. 윤 당선인은 “정부를 믿고 신뢰할 때 국민통합이 가능하다”며 “새 정부는 일 잘하는 정부, 능력과 실력을 겸비한 정부가 돼야 한다”고 말했다. 그는 “책상에서가 아닌 늘 현장에 중심을 두고 현장 목소리를 최대한 반영해달라”고도 했다.'

te = te.split('다.')
temp = []
for t in te[:-1] :
  temp.append(t+'다.')
temp.append(te[-1])

temp = remove_text(temp)
temp = text_splicing(temp, max_len) 
temp

['윤 당선인은 이날 현판식 직후 인수위 첫 전체회의를 주재하고 “국정과제 수립에 있어 국가의 안보, 국민의 민생에 한 치의 빈틈이 없어야 한다”며 이같이 강조했다.',
 ' 이어 “국정과제 우선순위 설정은 궁극적으로 국민통합을 위한 것”이라며 “국민께서 어느 지역에 사느냐와 관계없이 공정한 기회를 보장받아야 한다”고 했다.',
 ' 윤 당선인은 “정부를 믿고 신뢰할 때 국민통합이 가능하다”며 “새 정부는 일 잘하는 정부, 능력과 실력을 겸비한 정부가 돼야 한다”고 말했다.',
 ' 그는 “책상에서가 아닌 늘 현장에 중심을 두고 현장 목소리를 최대한 반영해달라”고도 했다.']

In [ ]:
# page 정보 ( 기사파일명을 dic_article에 담기 )
testset = []
file_num = 0

for (path, dir, files) in os.walk(testdata_path):
  for filename in files :
    if filename[-6:] == '00.xml':   # 페이지 정보
        continue
    
    elif filename[-4:] == '.xml':
      try : 
        with open("%s/%s" % (path, filename), "r", encoding="utf-8") as f:
          soup = BeautifulSoup(f, "html.parser")
          testset.append(soup.find("headline").text)
          testset.append(soup.find("subheadline").text)
          
          t = soup.find("datacontent").text
          t = t.split('다.')
          temp = []
          for tt in t[:-1] :
            temp.append(tt+'다.')
          temp.append(t[-1])
          temp = remove_text(temp)
          temp = text_splicing(temp, max_len) 

          testset.extend(temp)
          file_num += 1

      except:
        print('not exist ', "%s/%s" % (testdata_path, filename))  # 지면광고, 전면광고

print("len: ", len(testset))
print(file_num)
testset[0:10]

not exist  /content/drive/Othercomputers/내 노트북/신문원문PDF/01100611.20170102000010107.xml
not exist  /content/drive/Othercomputers/내 노트북/신문원문PDF/01100611.20170102000010106.xml
not exist  /content/drive/Othercomputers/내 노트북/신문원문PDF/01100611.20170102000010105.xml
not exist  /content/drive/Othercomputers/내 노트북/신문원문PDF/01100611.20170102000020206.xml
not exist  /content/drive/Othercomputers/내 노트북/신문원문PDF/01100611.20170102000030304.xml
not exist  /content/drive/Othercomputers/내 노트북/신문원문PDF/01100611.20170102000070701.xml
not exist  /content/drive/Othercomputers/내 노트북/신문원문PDF/01100611.20170102000060608.xml
not exist  /content/drive/Othercomputers/내 노트북/신문원문PDF/01100611.20170102000090905.xml
not exist  /content/drive/Othercomputers/내 노트북/신문원문PDF/01100611.20170102000141402.xml
not exist  /content/drive/Othercomputers/내 노트북/신문원문PDF/01100611.20170102000151503.xml
not exist 

['차기대통령 첫 덕목은 ‘소통과 통합’',
 '‘청렴·도덕성’ ‘강력 리더십’ 順 반기문·문재인 2강 이재명 1중',
 '조기 대선이 가시화된 가운데 차기대통령이 갖춰야 할 덕목으로 국민 3명 중 1명은 ‘소통 및 사회통합 능력’을꼽았다.',
 ' 차기 대선후보 선호도에서는반기문 전 유엔사무총장(21.7%)과 문재인 전 더불어민주당 대표(18.5%)가오차범위 내 접전인 가운데 이재명 성남시장(11.5%)이 뒤를 쫓는 ‘2강 1중’구도로 나타났다.',
 '1일 서울신문이 새해를 맞아 에이스리서치에 의뢰해 지난달 28~29일 19세이상 남녀 1009명을 대상으로 벌인 여론조사(표본오차 95% 신뢰수준에서±3.1% 포인트)에 따르면 차기 대통령이 갖춰야 할 덕목으로 ‘소통 및 사회통합 능력’(34.3%), ‘청렴성 및 도덕성’(24.8%)이 우선 꼽혔다.',
 ' 이런 덕목은일방통행식 국정 운영과 최순실 국정 농단 등 박근혜 대통령의 탄핵 사유와 밀접한 관련이 있다는 점에서 차기 대선구도와 맞물려 시사하는 바가 크다.',
 '특히 올 경제성장률이 외환위기 이후 20년 만에 2%(정부 2.6%)로 전망되는 등 최악의 위기 상황임에도 ‘강력한 리더십’(13.4%)이나 ‘경제활성화능력’(12.5%)은 후순위였고 ‘정치 경험 및 경륜’(6.4%), ‘외교·안보·통일전문성’(4.5%)에 대한 갈증도 미미했다는 점은 주목할 만하다.',
 '2강 1중을 잇는 여야 차기 대선후보는 안철수 국민의당 전 공동대표(5.7%), 박원순 서울시장(3.0%), 손학규전 민주당 대표(2.1%) 순으로 나타났다.',
 ' 반 전 총장이 범여권 후보로 나서고민주당 문 전 대표와 국민의당 안 전 대표가 ‘가상 3자대결’을 벌인다면 반 전총장과 문 전 대표가 각각 31.1%와 30.4%로 0.7% 포인트 차이로 초박빙 양상으로 조사됐다.',
 ' 안 전 대표는 11.3%에 그쳤다.']

# 2. 데이터 전처리

+ [KoBertTokenizer](#KoBertTokenizer)
+ [CorpusDataset](#CorpusDataset)
+ [Preprocessor](#Preprocessor)
+ [Config](#Config)

## KoBertTokenizer

[github](https://github.com/monologg/KoBERT-Transformers/blob/master/kobert_transformers/tokenization_kobert.py)

In [9]:
import logging
import os
import unicodedata
from shutil import copyfile

from transformers import PreTrainedTokenizer

logger = logging.getLogger(__name__)

VOCAB_FILES_NAMES = {
    "vocab_file": "tokenizer_78b3253a26.model",
    "vocab_txt": "vocab.txt",
}

PRETRAINED_VOCAB_FILES_MAP = {
    "vocab_file": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/tokenizer_78b3253a26.model",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/tokenizer_78b3253a26.model",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/tokenizer_78b3253a26.model",
    },
    "vocab_txt": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/vocab.txt",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/vocab.txt",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/vocab.txt",
    },
}

PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES = {
    "monologg/kobert": 512,
    "monologg/kobert-lm": 512,
    "monologg/distilkobert": 512,
}

PRETRAINED_INIT_CONFIGURATION = {
    "monologg/kobert": {"do_lower_case": False},
    "monologg/kobert-lm": {"do_lower_case": False},
    "monologg/distilkobert": {"do_lower_case": False},
}

SPIECE_UNDERLINE = "▁"


class KoBertTokenizer(PreTrainedTokenizer):
    """
    SentencePiece based tokenizer. Peculiarities:
        - requires `SentencePiece <https://github.com/google/sentencepiece>`_
    """

    vocab_files_names = VOCAB_FILES_NAMES
    pretrained_vocab_files_map = PRETRAINED_VOCAB_FILES_MAP
    pretrained_init_configuration = PRETRAINED_INIT_CONFIGURATION
    max_model_input_sizes = PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES

    def __init__(
        self,
        vocab_file,
        vocab_txt,
        do_lower_case=False,
        remove_space=True,
        keep_accents=False,
        unk_token="[UNK]",
        sep_token="[SEP]",
        pad_token="[PAD]",
        cls_token="[CLS]",
        mask_token="[MASK]",
        **kwargs,
    ):
        super().__init__(
            unk_token=unk_token,
            sep_token=sep_token,
            pad_token=pad_token,
            cls_token=cls_token,
            mask_token=mask_token,
            **kwargs,
        )

        # Build vocab
        self.token2idx = dict()
        self.idx2token = []
        with open(vocab_txt, "r", encoding="utf-8") as f:
            for idx, token in enumerate(f):
                token = token.strip()
                self.token2idx[token] = idx
                self.idx2token.append(token)

        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning(
                "You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                "pip install sentencepiece"
            )

        self.do_lower_case = do_lower_case
        self.remove_space = remove_space
        self.keep_accents = keep_accents
        self.vocab_file = vocab_file
        self.vocab_txt = vocab_txt

        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(vocab_file)

    @property
    def vocab_size(self):
        return len(self.idx2token)

    def get_vocab(self):
        return dict(self.token2idx, **self.added_tokens_encoder)

    def __getstate__(self):
        state = self.__dict__.copy()
        state["sp_model"] = None
        return state

    def __setstate__(self, d):
        self.__dict__ = d
        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning(
                "You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                "pip install sentencepiece"
            )
        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(self.vocab_file)

    def preprocess_text(self, inputs):
        if self.remove_space:
            outputs = " ".join(inputs.strip().split())
        else:
            outputs = inputs
        outputs = outputs.replace("``", '"').replace("''", '"')

        if not self.keep_accents:
            outputs = unicodedata.normalize("NFKD", outputs)
            outputs = "".join([c for c in outputs if not unicodedata.combining(c)])
        if self.do_lower_case:
            outputs = outputs.lower()

        return outputs

    def _tokenize(self, text):
        """Tokenize a string."""
        text = self.preprocess_text(text)
        pieces = self.sp_model.encode(text, out_type=str)
        new_pieces = []
        for piece in pieces:
            if len(piece) > 1 and piece[-1] == str(",") and piece[-2].isdigit():
                cur_pieces = self.sp_model.EncodeAsPieces(piece[:-1].replace(SPIECE_UNDERLINE, ""))
                if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
                    if len(cur_pieces[0]) == 1:
                        cur_pieces = cur_pieces[1:]
                    else:
                        cur_pieces[0] = cur_pieces[0][1:]
                cur_pieces.append(piece[-1])
                new_pieces.extend(cur_pieces)
            else:
                new_pieces.append(piece)

        return new_pieces

    def _convert_token_to_id(self, token):
        """ Converts a token (str/unicode) in an id using the vocab. """
        return self.token2idx.get(token, self.token2idx[self.unk_token])

    def _convert_id_to_token(self, index):
        """Converts an index (integer) in a token (string/unicode) using the vocab."""
        return self.idx2token[index]

    def convert_tokens_to_string(self, tokens):
        """Converts a sequence of tokens (strings for sub-words) in a single string."""
        out_string = "".join(tokens).replace(SPIECE_UNDERLINE, " ").strip()
        return out_string

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        """
        Build model inputs from a sequence or a pair of sequence for sequence classification tasks
        by concatenating and adding special tokens.
        A KoBERT sequence has the following format:
            single sequence: [CLS] X [SEP]
            pair of sequences: [CLS] A [SEP] B [SEP]
        """
        if token_ids_1 is None:
            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]
        cls = [self.cls_token_id]
        sep = [self.sep_token_id]
        return cls + token_ids_0 + sep + token_ids_1 + sep

    def get_special_tokens_mask(self, token_ids_0, token_ids_1=None, already_has_special_tokens=False):
        """
        Retrieves sequence ids from a token list that has no special tokens added. This method is called when adding
        special tokens using the tokenizer ``prepare_for_model`` or ``encode_plus`` methods.
        Args:
            token_ids_0: list of ids (must not contain special tokens)
            token_ids_1: Optional list of ids (must not contain special tokens), necessary when fetching sequence ids
                for sequence pairs
            already_has_special_tokens: (default False) Set to True if the token list is already formated with
                special tokens for the model
        Returns:
            A list of integers in the range [0, 1]: 0 for a special token, 1 for a sequence token.
        """

        if already_has_special_tokens:
            if token_ids_1 is not None:
                raise ValueError(
                    "You should not supply a second sequence if the provided sequence of "
                    "ids is already formated with special tokens for the model."
                )
            return list(
                map(
                    lambda x: 1 if x in [self.sep_token_id, self.cls_token_id] else 0,
                    token_ids_0,
                )
            )

        if token_ids_1 is not None:
            return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]
        return [1] + ([0] * len(token_ids_0)) + [1]

    def create_token_type_ids_from_sequences(self, token_ids_0, token_ids_1=None):
        """
        Creates a mask from the two sequences passed to be used in a sequence-pair classification task.
        A KoBERT sequence pair mask has the following format:
        0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
        | first sequence    | second sequence
        if token_ids_1 is None, only returns the first portion of the mask (0's).
        """
        sep = [self.sep_token_id]
        cls = [self.cls_token_id]
        if token_ids_1 is None:
            return len(cls + token_ids_0 + sep) * [0]
        return len(cls + token_ids_0 + sep) * [0] + len(token_ids_1 + sep) * [1]

    def save_vocabulary(self, save_directory):
        """Save the sentencepiece vocabulary (copy original file) and special tokens file
        to a directory.
        """
        if not os.path.isdir(save_directory):
            logger.error("Vocabulary path ({}) should be a directory".format(save_directory))
            return

        # 1. Save sentencepiece model
        out_vocab_model = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_file"])

        if os.path.abspath(self.vocab_file) != os.path.abspath(out_vocab_model):
            copyfile(self.vocab_file, out_vocab_model)

        # 2. Save vocab.txt
        index = 0
        out_vocab_txt = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_txt"])
        with open(out_vocab_txt, "w", encoding="utf-8") as writer:
            for token, token_index in sorted(self.token2idx.items(), key=lambda kv: kv[1]):
                if index != token_index:
                    logger.warning(
                        "Saving vocabulary to {}: vocabulary indices are not consecutive."
                        " Please check that the vocabulary is not corrupted!".format(out_vocab_txt)
                    )
                    index = token_index
                writer.write(token + "\n")
                index += 1

        return out_vocab_model, out_vocab_txt

In [ ]:
# encode test
# tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
# tokenizer.encode("한국어 모델을 공유합니다.")

Downloading:   0%|          | 0.00/363k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/76.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


## CorpusDataset

In [10]:
class CorpusDataset(Dataset):
    def __init__(self, sentences, transform: Callable[[List, List], Tuple]):
        self.sentences = []
        self.slot_labels = ["UNK", "PAD", "B", "I"]
        self.transform = transform
        self._load_data(sentences)

    def _load_data(self, sentences):
        """data를 file에서 불러온다.

        Args:
            data_path: file 경로
        """
        self.sentences = [sen.split() for sen in sentences]
        # with open(data_path, mode="r", encoding="utf-8") as f:
        #     lines = f.readlines()
        #     self.sentences = [line.split() for line in lines]

    def _get_tags(self, sentence: List[str]) -> List[str]:
        """문장에 대해 띄어쓰기 tagging을 한다.
        character 단위로 분리하여 BI tagging을 한다.

        Args:
            sentence: 문장

        Retrns:
            문장의 각 토큰에 대해 tagging한 결과 리턴
            ["B", "I"]
        """

        tags = []
        for word in sentence:
            for i in range(len(word)):
                if i == 0:
                    tags.append("B")
                else:
                    tags.append("I")
        return tags

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = "".join(self.sentences[idx])
        sentence = [s for s in sentence]
        tags = self._get_tags(self.sentences[idx])
        tags = [self.slot_labels.index(t) for t in tags]

        (
            input_ids,
            attention_mask,
            token_type_ids,
            slot_label_ids, 
        ) = self.transform(sentence, tags)

        return input_ids, attention_mask, token_type_ids, slot_label_ids

## Preprocessor

In [11]:
from typing import List, Tuple

class Preprocessor :
    def __init__(self, max_len: int):
        self.tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert")
        self.max_len = max_len
        self.pad_token_id = 0

    def get_input_features(self, sentence, tags
    ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
        """문장과 띄어쓰기 tagging에 대해 feature로 변환한다.

        Args:
            sentence: 문장
            tags: 띄어쓰기 tagging

        Returns:
            feature를 리턴한다.
            input_ids, attention_mask, token_type_ids, slot_labels
        """

        input_tokens = []
        slot_label_ids = []
					
        # tokenize
        for word, tag in zip(sentence, tags):
            tokens = self.tokenizer.tokenize(word)

            if len(tokens) == 0:
                tokens = self.tokenizer.unk_token

            input_tokens.extend(tokens)

            for i in range(len(tokens)):
                if i == 0:
                    slot_label_ids.extend([tag])
                else:
                    slot_label_ids.extend([self.pad_token_id])

        # max_len보다 길이가 길면 뒤에 자르기
        if len(input_tokens) > self.max_len - 2:
            input_tokens = input_tokens[: self.max_len - 2]
            slot_label_ids = slot_label_ids[: self.max_len - 2]

        # cls, sep 추가
        input_tokens = (
            [self.tokenizer.cls_token] + input_tokens + [self.tokenizer.sep_token]
        )
        slot_label_ids = [self.pad_token_id] + slot_label_ids + [self.pad_token_id]

        # token을 id로 변환
        input_ids = self.tokenizer.convert_tokens_to_ids(input_tokens)

        attention_mask = [1] * len(input_ids)
        token_type_ids = [0] * len(input_ids)

        # padding
        pad_len = self.max_len - len(input_tokens)
        input_ids = input_ids + ([self.tokenizer.pad_token_id] * pad_len)
        slot_label_ids = slot_label_ids + ([self.pad_token_id] * pad_len)
        attention_mask = attention_mask + ([0] * pad_len)
        token_type_ids = token_type_ids + ([0] * pad_len)

        input_ids = torch.tensor(input_ids, dtype=torch.long)
        attention_mask = torch.tensor(attention_mask, dtype=torch.long)
        token_type_ids = torch.tensor(token_type_ids, dtype=torch.long)
        slot_label_ids = torch.tensor(slot_label_ids, dtype=torch.long)

        return input_ids, attention_mask, token_type_ids, slot_label_ids

## Config

파라미터 정의
yaml파일 대신에 class 사용

In [12]:
# yaml 파일 대신에 객체로 생성
class Config():
  def __init__(self) :
    self.task= 'korean_spacing_20210101'
    self.log_path= data_path+'/logs'
    self.bert_model =  'monologg/kobert'
    self.train_data_path= data_path+'/train.txt'
    self.val_data_path= data_path+'/val.txt'
    self.test_data_path= data_path+'/test.txt'
    self.max_len= 128
    self.train_batch_size= 64
    self.eval_batch_size= 64
    self.dropout_rate= 0.1
    self.gpus= torch.cuda.device_count()

# 3. 모델



## SpacingBertModel

In [13]:
from math import log
class SpacingBertModel(pl.LightningModule):
    def __init__(self, config, dataset):
        super().__init__()
        self.config = config
        self.dataset = dataset
        self.slot_labels_type = ["UNK", "PAD", "B", "I"]
        self.pad_token_id = 0

        self.bert_config = BertConfig.from_pretrained(
            self.config.bert_model, num_labels=len(self.slot_labels_type)
        )
        self.model = BertModel.from_pretrained(
            self.config.bert_model, config=self.bert_config
        )
        self.dropout = nn.Dropout(self.config.dropout_rate)
        self.linear = nn.Linear(
            self.bert_config.hidden_size, len(self.slot_labels_type)
        )

    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        return self.linear(self.dropout(outputs[0]))

    def training_step(self, batch, batch_nb):

        input_ids, attention_mask, token_type_ids, slot_label_ids = batch

        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        loss = self._calculate_loss(outputs, slot_label_ids) # slot_labels : labels
        pred_slot_labels, gt_slot_labels = self._convert_ids_to_labels(
            outputs, slot_label_ids
        )

        f1 = self._f1_score(gt_slot_labels, pred_slot_labels)
        acc = self._calculate_accuracy(outputs, slot_label_ids)

        return {"loss": loss, "f1": f1, "acc": acc, "log": {"train_loss": loss, "train_f1": f1}}

    def training_end(self, batch, batch_nb):

        input_ids, attention_mask, token_type_ids, slot_label_ids = batch

        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        loss = self._calculate_loss(outputs, slot_label_ids) # slot_labels : labels

        pred_slot_labels, gt_slot_labels = self._convert_ids_to_labels(
            outputs, slot_label_ids
        )
        f1 = self._f1_score(gt_slot_labels, pred_slot_labels)
        acc = self._calculate_accuracy(outputs, slot_label_ids)

        print("training_end's loss : ", loss)
        print("training_end's acc : ", acc)
        print("training_end's f1 : ", f1)

        return {"loss": loss, "f1": f1, "acc": acc, "log": {"train_loss": loss, "train_f1": f1}}

    def training_epoch_end(self, outputs):
        super().on_train_epoch_end()
        print("training_epoch_end")


    def validation_step(self, batch, batch_nb):

        input_ids, attention_mask, token_type_ids, slot_label_ids = batch

        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        loss = self._calculate_loss(outputs, slot_label_ids)
        pred_slot_labels, gt_slot_labels = self._convert_ids_to_labels(
            outputs, slot_label_ids
        )

        val_f1 = self._f1_score(gt_slot_labels, pred_slot_labels)
        val_acc = self._calculate_accuracy(outputs, slot_label_ids)
        return {"val_loss": loss, "val_f1": val_f1, 'val_acc':val_acc}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        val_f1 = torch.stack([x['val_f1'] for x in outputs]).mean()
        val_acc = torch.stack([x['val_acc'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss, 'val_f1':val_f1, 'val_acc':val_acc}
        print('validation_epoch_end : ', tensorboard_logs)
        return {'val_acc':val_acc, 'val_loss': avg_loss, 'val_f1':val_f1, 'log': tensorboard_logs}
    
    def validation_end(self, outputs):
        val_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        val_acc = torch.stack([x["val_acc"] for x in outputs]).mean()
        val_f1 = torch.stack([x["val_f1"] for x in outputs]).mean()
        tensorboard_logs = {
            "val_loss": val_loss,
            "val_f1": val_f1,
            "val_acc" : val_acc
        }
        print("validation_end : ", tensorboard_logs)
        return {'val_acc':val_acc, 'val_loss': val_loss, 'val_f1':val_f1, 'log': tensorboard_logs}
    
    def test_step(self, batch, batch_nb):
        input_ids, attention_mask, token_type_ids, slot_label_ids = batch
        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        pred_slot_labels, gt_slot_labels = self._convert_ids_to_labels(
            outputs, slot_label_ids
        )
        f1 = self._f1_score(gt_slot_labels, pred_slot_labels)
        acc = self._calculate_accuracy(outputs, slot_label_ids)
        loss = self._calculate_loss(outputs, slot_label_ids)
        return {"input_ids":input_ids, "pred_slot_labels" : pred_slot_labels, "test_loss": loss, "test_f1": f1, "test_acc": acc}

    def test_end(self, outputs):
        test_f1 = torch.stack([x["test_f1"] for x in outputs]).mean()
        test_loss = torch.stack([x["test_loss"] for x in outputs]).mean()
        test_acc = torch.stack([x["test_acc"] for x in outputs]).mean()
        self.log("test_loss", test_loss)
        self.log("test_acc", test_acc)
        self.log("test_f1", test_f1)
        print('test_end')
        return {"pred_slot_labels" : [x["pred_slot_labels"] for x in outputs], "test_loss": test_loss, "test_f1": test_f1, "test_acc": test_acc}
    
    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        f1 = torch.stack([x['test_f1'] for x in outputs]).mean()
        acc = torch.stack([x['test_acc'] for x in outputs]).mean()
        tensorboard_logs = {'test_loss': avg_loss, 'test_f1':f1, 'test_acc':acc}
        print('test_epoch_end : ', tensorboard_logs)
        return {'test_acc':acc, 'test_loss': avg_loss, 'test_f1':f1, 'log': tensorboard_logs}

    def predict_step(self, batch, batch_idx, dataloader_idx=0):   # prediction : forward(), predict_step()
        input_ids, attention_mask, token_type_ids, slot_label_ids = batch    # slot_label은 없음.
        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        pred_slot_labels, gt_slot_labels = self._convert_ids_to_labels(
            outputs, slot_label_ids
        )
        return {'pred_slot_labels':pred_slot_labels, 'gt_slot_labels': gt_slot_labels}


    def configure_optimizers(self):
        return AdamW(self.model.parameters(), lr=2e-5, eps=1e-8)

    def train_dataloader(self):
        return DataLoader(self.dataset["train"], batch_size=self.config.train_batch_size)

    def val_dataloader(self):
        return DataLoader(self.dataset["val"], batch_size=self.config.eval_batch_size)

    def test_dataloader(self):
        return DataLoader(self.dataset["test"], batch_size=self.config.eval_batch_size)

    def pred_dataloader(self, dataset):
        return DataLoader(dataset, batch_size=1)

    def _calculate_loss(self, outputs, labels):
        active_logits = outputs.view(-1, len(self.slot_labels_type))
        active_labels = labels.view(-1)
        loss = F.cross_entropy(active_logits, active_labels)
        return loss
        
    def _calculate_accuracy(self, outputs, labels):
        active_logits = outputs.view(-1, len(self.slot_labels_type))
        active_labels = labels.view(-1)
        accuracy = accuracy_score(active_logits, active_labels)
        return accuracy

    def _f1_score(self, gt_slot_labels, pred_slot_labels):
        return torch.tensor(
            f1_score(gt_slot_labels, pred_slot_labels), dtype=torch.float32
        )

    def _convert_ids_to_labels(self, outputs, slot_labels):
        _, y_hat = torch.max(outputs, dim=2)
        y_hat = y_hat.detach().cpu().numpy()
        slot_label_ids = slot_labels.detach().cpu().numpy()

        slot_label_map = {i: label for i, label in enumerate(self.slot_labels_type)}
        slot_gt_labels = [[] for _ in range(slot_label_ids.shape[0])]
        slot_pred_labels = [[] for _ in range(slot_label_ids.shape[0])]

        for i in range(slot_label_ids.shape[0]):
            for j in range(slot_label_ids.shape[1]):
                if slot_label_ids[i, j] != self.pad_token_id:
                    slot_gt_labels[i].append(slot_label_map[slot_label_ids[i][j]])
                    slot_pred_labels[i].append(slot_label_map[y_hat[i][j]])

        return slot_pred_labels, slot_gt_labels

# 4. 학습

+ Preprocessor 선언, dataset 생성, 모델 선언
+ callbacks : TensorBoardLogger, ModelCheckpoint, EarlyStopping, LearningRateMonitor

## Preprocessor 선언, dataset 생성, 모델 선언

In [23]:
config = Config()
preprocessor = Preprocessor(config.max_len)

Downloading:   0%|          | 0.00/363k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/76.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [ ]:
dataset = {}
dataset["train"] = CorpusDataset(train, preprocessor.get_input_features)
dataset["val"] = CorpusDataset(val, preprocessor.get_input_features)
dataset["test"] = CorpusDataset(testset, preprocessor.get_input_features)

bert_finetuner = SpacingBertModel(config, dataset)

Downloading:   0%|          | 0.00/352M [00:00<?, ?B/s]

In [ ]:
bert_finetuner

## callbacks

In [ ]:
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.callbacks import LearningRateMonitor

logger = TensorBoardLogger(
    save_dir=os.path.join(config.log_path, config.task), version=3, name=config.task
)

acc_checkpoint_callback = ModelCheckpoint(
    dirpath=data_path+'/checkpoints/'+ config.task, 
    filename="acc_{epoch}_{val_loss:35f}",
    verbose=True,
    monitor='val_acc',
    mode='max',
    save_top_k=3,
    save_last=True)

loss_checkpoint_callback = ModelCheckpoint(
    dirpath=data_path+'/checkpoints/'+ config.task, 
    filename="val_{epoch}_{val_acc:.3f}",
    verbose=True,
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_last=True)

loss_early_stop_callback = EarlyStopping(
    monitor='val_loss',
    min_delta=0.0001,
    patience=3,
    verbose=True,
    mode='min'
)

acc_early_stop_callback = EarlyStopping(
    monitor='val_acc',
    min_delta=0.0001,
    patience=3,
    verbose=True,
    mode='max'
)
lrmonitor_callback = LearningRateMonitor(logging_interval='step')

## Train

In [ ]:
trainer = pl.Trainer(
    gpus=config.gpus,
    callbacks=[acc_checkpoint_callback, loss_checkpoint_callback, lrmonitor_callback],
    logger=logger,
    max_epochs=20,
)

trainer.fit(bert_finetuner)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,

  | Name    | Type      | Params
--------------------------------------
0 | model   | BertModel | 92.2 M
1 | dropout | Dropout   | 0     
2 | linear  | Linear    | 3.1 K 
--------------------------------------
92.2 M    Trainable params
0         Non-trainable params
92.2 M    Total params
368.760   Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Checkpoint directory /content/drive/My Drive/Colab Notebooks/NextLab/news_class9x1400/checkpoin

Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


validation_epoch_end :  {'val_loss': tensor(0.0311, device='cuda:0'), 'val_f1': tensor(0.9185)}


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


...

Epoch 9, global step 35819: val_loss was not in top 1


Validating: 0it [00:00, ?it/s]

Epoch 10, global step 39401: val_acc was not in top 3


validation_epoch_end :  {'val_loss': tensor(0.0308, device='cuda:0'), 'val_f1': tensor(0.9300)}
training_epoch_end


Epoch 10, global step 39401: val_loss was not in top 1


In [ ]:
# test의 label이 부정확하기 때문에 test 대신 pred를 통해 확인

# test = trainer.test(model=bert_finetuner, verbose=True, dataloaders=bert_finetuner.test_dataloader())

In [ ]:
# torch.save(bert_finetuner, data_path+'/model_128_batch_version3_torch.pt')    # model 전체 저장

## 학습 이어서하기

끊긴 모델 훈련 이어서 하기

trainer에 epochs를 재설정하고 시작  

<br>

참고 :

  - [PyTorch에서 훈련 된 모델을 저장하는 가장 좋은 방법은?](http://daplus.net/python-pytorch%EC%97%90%EC%84%9C-%ED%9B%88%EB%A0%A8-%EB%90%9C-%EB%AA%A8%EB%8D%B8%EC%9D%84-%EC%A0%80%EC%9E%A5%ED%95%98%EB%8A%94-%EA%B0%80%EC%9E%A5-%EC%A2%8B%EC%9D%80-%EB%B0%A9%EB%B2%95%EC%9D%80/)

  - [pytorch lightening 모델 저장](https://brunch.co.kr/@yysttong/90)

  - [모델 체크포인트(.ckpt)를 어떻게 로드하고 사용하나요?](https://forums.pytorchlightning.ai/t/how-to-load-and-use-model-checkpoint-ckpt/677)

In [ ]:
# state = torch.load(data_path+'/checkpoints/'+ config.task + '/last.ckpt' )  # torch로 모델 받아오기
model2 = SpacingBertModel(config, dataset).load_from_checkpoint(data_path+'/checkpoints/'+ config.task + '/last.ckpt', config=config, dataset=dataset )   # checkpoint로 모델 받아오기
type(model2), bert_finetuner.state_dict()

In [ ]:
trainer = pl.Trainer(gpus=config.gpus, max_epochs=2)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
trainer.fit(model2)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,

  | Name    | Type      | Params
--------------------------------------
0 | model   | BertModel | 92.2 M
1 | dropout | Dropout   | 0     
2 | linear  | Linear    | 3.1 K 
--------------------------------------
92.2 M    Trainable params
0         Non-trainable params
92.2 M    Total params
368.760   Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Checkpoint directory /content/drive/My Drive/Colab Notebooks/NextLab/news_class9x1400/checkpoints/korean_spacing_20210101 exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists 

Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


validation_epoch_end :  {'val_loss': tensor(0.0311, device='cuda:0'), 'val_f1': tensor(0.9342)}


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


...

Epoch 0, global step 3710: val_loss was not in top 1


Validating: 0it [00:00, ?it/s]

Epoch 1, global step 7292: val_acc was not in top 3


validation_epoch_end :  {'val_loss': tensor(0.0321, device='cuda:0'), 'val_f1': tensor(0.9289)}
training_epoch_end


Epoch 1, global step 7292: val_loss was not in top 1
Saving latest checkpoint...
Saving latest checkpoint...


In [ ]:
torch.save(model2, data_path+'/model_128_batch_version3_torch_0319.pt')    # model 전체 저장

# 예측

## 기사 원문의 띄어쓰기가 맞는지 확인

xml파일의 문장의 띄어쓰기가 맞는지 확인

- bertmodel의 prediction 부분을 추가해서 가중치를 받은 모델을 새로 생성하였습니다.

- 아래 dataset['test']는 예측값을 얻을 데이터셋입니다.  

- 예측값을 get_dataset_to_check_spacing(dataset) 함수에 dataset만 입력하면 얻을 수 있습니다.

In [ ]:
dataset = {}
dataset["test"] = CorpusDataset(testset, preprocessor.get_input_features)

In [ ]:
# 마지막 checkout의 weight를 가져와서 모델의 가중치로 적용
model2 = SpacingBertModel(config, dataset).load_from_checkpoint(data_path+'/checkpoints/'+ config.task + '/last.ckpt', config=config, dataset=dataset )
trainer = pl.Trainer(gpus=config.gpus)
type(model2)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


__main__.SpacingBertModel

In [ ]:
predictions = trainer.predict(model2, dataloaders=model2.pred_dataloader(dataset["test"]))

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Predicting: 0it [00:00, ?it/s]

In [ ]:
len(dataset['test']), len(predictions)

(2048, 2048)

In [ ]:
list_string = []
list_pred_slot_label = []
for t, p_dic in zip(testset, predictions) :
  list_pred_slot_label.append(p_dic['pred_slot_labels'][0])   # 띄어쓰기 예측한 값
  s = ''
  for tt, v in zip(t.replace(' ', ''), p_dic['pred_slot_labels'][0]) :
    s += ' ' + tt if v == 'B' else tt

  s = s.lstrip()
  list_string.append(s)

print(list_string[2])
len(list_string), len(list_pred_slot_label)

조기 대선이 가시화된 가운데 차기 대통령이 갖춰야 할 덕목으로 국민 3명 중 1명은 ‘소통 및 사회통합 능력’을 꼽았다.


(2048, 2048)

In [ ]:
import pandas as pd

pred_dataset = pd.DataFrame({'input_data' : testset, 'pred_slot_label' : list_pred_slot_label, 'pred_content' : list_string })
pred_dataset.head()

,input_data,pred_slot_label,pred_content
0,차기대통령 첫 덕목은 ‘소통과 통합’,"[B, I, B, I, I, B, B, I, I, B, I, I, I, B, I, I]",차기 대통령 첫 덕목은 ‘소통과 통합’
1,‘청렴·도덕성’ ‘강력 리더십’ 順 반기문·문재인 2강 이재명 1중,"[B, I, I, I, I, I, I, I, B, I, I, I, I, I, I, ...",‘청렴·도덕성’ ‘강력리더십’ 順 반기문·문재인 2강 이재명 1중
2,조기 대선이 가시화된 가운데 차기대통령이 갖춰야 할 덕목으로 국민 3명 중 1명은 ...,"[B, I, B, I, I, B, I, I, I, B, I, I, B, I, B, ...",조기 대선이 가시화된 가운데 차기 대통령이 갖춰야 할 덕목으로 국민 3명 중 1명은...
3,차기 대선후보 선호도에서는반기문 전 유엔사무총장(21.7%)과 문재인 전 더불어민...,"[B, I, B, I, I, I, B, I, I, I, I, I, B, I, I, ...",차기 대선후보 선호도에서는 반기문 전 유엔사무총장(21.7%)과 문재인 전 더불어민...
4,1일 서울신문이 새해를 맞아 에이스리서치에 의뢰해 지난달 28~29일 19세이상 남...,"[B, I, B, I, I, I, I, B, I, I, B, I, B, I, I, ...",1일 서울신문이 새해를 맞아 에이스리서치에 의뢰해 지난달 28~29일 19세 이상 ...


In [ ]:
pred_dataset.to_csv("result_spacing_predictions.csv", mode='w')

In [19]:
# 입력값 : list, 리턴값 : pd.DataFrame
def get_dataset_to_check_spacing(dataset) :
  model2 = SpacingBertModel(config, dataset).load_from_checkpoint(data_path+'/checkpoints/'+ config.task + '/last.ckpt', config=config, dataset=dataset )
  trainer = pl.Trainer(gpus=config.gpus)
  predictions = trainer.predict(model2, dataloaders=model2.pred_dataloader(CorpusDataset(dataset, preprocessor.get_input_features)))  # dataset으로 예측

  list_string = []
  list_pred_slot_label = []
  for t, p_dic in zip(dataset, predictions) :
    list_pred_slot_label.append(p_dic['pred_slot_labels'][0])   # 띄어쓰기 예측한 값
    s = ''
    for tt, v in zip(t.replace(' ', ''), p_dic['pred_slot_labels'][0]) :
      s += ' ' + tt if v == 'B' else tt
    s = s.lstrip()
    list_string.append(s)
  print("총 lines : " ,len(dataset), len(list_pred_slot_label), len(list_string))
  return list_pred_slot_label, list_string

## pdf에서 text 긁어오기 ( 실제 데이터 )


이제, 우리의 목표인 행의 마지막 단어와 그 다음 행의 앞 단어 사이에 띄어쓰기가 필요한지 여부를 알아볼 차례입니다.

In [15]:
from google.colab import files
uploaded = files.upload()
# uploaded

Saving news_test.txt to news_test (2).txt


In [16]:
f = open('news_test.txt', 'r')
lines = f.readlines()
lines

['독일 뮌헨에서 북쪽으로 200㎞ 떨어\n',
 '진 암베르크는 인구 4만 4000명의 작\n',
 '은 도시다. 하지만 앙겔라 메르켈 독일\n',
 '총리는 물론 전 세계 기업인과 학자들\n',
 '이 4차 산업혁명을 공부하기 위해 찾\n',
 '는 곳이다. 세계적인 전기전자 기업 지\n',
 '멘스가 자랑하는 스마트 공장이 이곳\n',
 '에 있기 때문이다. 이 공장은 어떤 비밀\n',
 '이 있기에 4차 산업혁명의 메카로 불\n',
 '리는 걸까.\n',
 '암베르크 공장은 커다란 병원 수술\n',
 '실 같았다. 안내자가 “건초 더미에서\n',
 '바늘을 찾는 게 더 쉬울 것”이라고 말\n',
 '할 정도로 먼지 한점 없이 깨끗했다. 축\n',
 '구장 1.5배인 1만㎡의 널찍한 공간에\n',
 '서 컨베이어벨트는 쉴 새 없이 돌아갔\n',
 '지만 ‘사람’은 거의 보이지 않았다. 의\n',
 '사 수술복과 비슷한 파란색 유니폼을\n',
 '입은 직원들이 드문드문 눈에 띄었는\n',
 '데, 모니터만 들여다보고 있을 뿐 뭔가\n',
 '를 만들거나 조립하지는 않았다. 암베\n',
 '르크 공장에선 전체 공정의 75%가 인\n',
 '간의 손이 필요 없는 자동화로 진행된\n',
 '다. 이 공장이 만드는 건 ‘시마틱 프로\n',
 '그램 가능 논리 제어 장치’(PLCs)로\n',
 '불리는 일종의 칩이다. 기계나 로봇을\n',
 '조종하고 움직이는 ‘두뇌’라고 생각하\n',
 '면 된다.\n',
 '암베르크·뮌헨(독일) 임주형기자\n',
 'hermes@seoul.co.kr\n',
 '▶관련기사 8면']

In [17]:
# 띄어쓰기가 필요한지 확인하기 위해서 line의 마지막 단어와 다음 line의 첫 단어 저장
 
words, word = [], []
for line in lines :
  line = line.strip()
  first, dot, last = line.find(' '), line.find('. '), line.rfind(' ')
  if dot == -1 :
    word.append( (line[:first], line[last:], len(line)) )
  else :
    word.append( (line[:first], 'FIN', len(line)))     # 줄의 마지막 ( dot 대신에 FIN)
    words.append(word)
    word = [('SRT', line[last:].strip(), len(line))]   # . 대신에 start, 줄의 시작 
words.append(word)

words, len(words)

([[('독일', ' 떨어', 20), ('진', ' 작', 22), ('은', 'FIN', 21)],
  [('SRT', '독일', 21), ('총리는', ' 학자들', 20), ('이', ' 찾', 20), ('는', 'FIN', 21)],
  [('SRT', '지', 21), ('멘스가', ' 이곳', 19), ('에', 'FIN', 22)],
  [('SRT', '비밀', 22),
   ('이', ' 불', 20),
   ('리는', ' 걸까.', 6),
   ('암베르크', ' 수술', 18),
   ('실', 'FIN', 20)],
  [('SRT', '더미에서', 20), ('바늘을', ' 말', 21), ('할', 'FIN', 22)],
  [('SRT', '축', 22), ('구장', ' 공간에', 21), ('서', ' 돌아갔', 20), ('지만', 'FIN', 22)],
  [('SRT', '의', 22),
   ('사', ' 유니폼을', 19),
   ('입은', ' 띄었는', 19),
   ('데,', ' 뭔가', 21),
   ('를', 'FIN', 20)],
  [('SRT', '암베', 20), ('르크', ' 인', 21), ('간의', ' 진행된', 20), ('다.', 'FIN', 22)],
  [('SRT', '프로', 22), ('그램', ' 장치’(PLCs)로', 22), ('불리는', 'FIN', 20)],
  [('SRT', '로봇을', 20),
   ('조종하고', ' 생각하', 20),
   ('면', ' 된다.', 5),
   ('암베르크·뮌헨(독일)', ' 임주형기자', 17),
   ('hermes@seoul.co.k', 'r', 18),
   ('▶관련기사', ' 8면', 8)]],
 10)

In [21]:
# 예측할 문자열 데이터셋 생성
text = ''.join(lines).replace('\n', '')
text_list = text.split('. ')
text_list = [text + '.' for text in text_list]
text_list, len(text_list)

(['독일 뮌헨에서 북쪽으로 200㎞ 떨어진 암베르크는 인구 4만 4000명의 작은 도시다.',
  '하지만 앙겔라 메르켈 독일총리는 물론 전 세계 기업인과 학자들이 4차 산업혁명을 공부하기 위해 찾는 곳이다.',
  '세계적인 전기전자 기업 지멘스가 자랑하는 스마트 공장이 이곳에 있기 때문이다.',
  '이 공장은 어떤 비밀이 있기에 4차 산업혁명의 메카로 불리는 걸까.암베르크 공장은 커다란 병원 수술실 같았다.',
  '안내자가 “건초 더미에서바늘을 찾는 게 더 쉬울 것”이라고 말할 정도로 먼지 한점 없이 깨끗했다.',
  '축구장 1.5배인 1만㎡의 널찍한 공간에서 컨베이어벨트는 쉴 새 없이 돌아갔지만 ‘사람’은 거의 보이지 않았다.',
  '의사 수술복과 비슷한 파란색 유니폼을입은 직원들이 드문드문 눈에 띄었는데, 모니터만 들여다보고 있을 뿐 뭔가를 만들거나 조립하지는 않았다.',
  '암베르크 공장에선 전체 공정의 75%가 인간의 손이 필요 없는 자동화로 진행된다.',
  '이 공장이 만드는 건 ‘시마틱 프로그램 가능 논리 제어 장치’(PLCs)로불리는 일종의 칩이다.',
  '기계나 로봇을조종하고 움직이는 ‘두뇌’라고 생각하면 된다.암베르크·뮌헨(독일) 임주형기자hermes@seoul.co.kr▶관련기사 8면.'],
 10)

In [23]:
preprocessor = Preprocessor(config.max_len)
list_pred_slot_label, pred_text_list = get_dataset_to_check_spacing(text_list)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Predicting: 0it [00:00, ?it/s]

총 lines :  10 10 10


In [24]:
pred, pred_idx, pred_len = pred_text_list[0], 0, len(pred_text_list)
answer = []
answer_text = []
for word in words :
  print(f'{pred_idx} line :')

  text = word[0][1]  # SRT
  start = 0
  for w in word[1:] :

    found = pred.find(text + w[0])
    if found == -1 :
      print(f"    not found : [{text+w[0]}]",pred) # 띄어쓰기 필요한 곳 
      answer.append(1)
      answer_text.append(text + ' ' + w[0])
    else :
      print(f"    found : [{text+w[0]}]",pred)
      answer.append(0)
      answer_text.append(text + w[0])
    
    start += w[2]
    if w[1] == 'FIN':
      pred_idx += 1
      if pred_idx == pred_len :
        break
      pred = pred_text_list[pred_idx]
    
    text = w[1]

# 1 line의 3번째 원문은 '세계 기업인과 학자들'이다.
# 띄어쓰기가 필요한 부분만 바꿔서 넣어야 정확도가 올라갈 것이다.

0 line :
    found : [ 떨어진] 독일 뮌헨에서 북쪽으로 200㎞ 떨어진 암베르크는 인구 4만4000명의 작은 도시다.
    found : [ 작은] 독일 뮌헨에서 북쪽으로 200㎞ 떨어진 암베르크는 인구 4만4000명의 작은 도시다.
1 line :
    not found : [독일총리는] 하지만 앙겔라 메르켈 독일 총리는 물론 전 세계 기업인 과 학자들이 4차 산업혁명을 공부하기 위해 찾는 곳이다.
    found : [ 학자들이] 하지만 앙겔라 메르켈 독일 총리는 물론 전 세계 기업인 과 학자들이 4차 산업혁명을 공부하기 위해 찾는 곳이다.
    found : [ 찾는] 하지만 앙겔라 메르켈 독일 총리는 물론 전 세계 기업인 과 학자들이 4차 산업혁명을 공부하기 위해 찾는 곳이다.
2 line :
    found : [지멘스가] 세계적인 전기전자 기업 지멘스가 자랑하는 스마트 공장이 이곳에 있기 때문이다.
    found : [ 이곳에] 세계적인 전기전자 기업 지멘스가 자랑하는 스마트 공장이 이곳에 있기 때문이다.
3 line :
    found : [비밀이] 이 공장은 어떤 비밀이 있기에 4차 산업혁명의 메카로 불리는 걸까. 암베르크 공장은 커다란 병원 수술실 같았다.
    found : [ 불리는] 이 공장은 어떤 비밀이 있기에 4차 산업혁명의 메카로 불리는 걸까. 암베르크 공장은 커다란 병원 수술실 같았다.
    not found : [ 걸까.암베르크] 이 공장은 어떤 비밀이 있기에 4차 산업혁명의 메카로 불리는 걸까. 암베르크 공장은 커다란 병원 수술실 같았다.
    found : [ 수술실] 이 공장은 어떤 비밀이 있기에 4차 산업혁명의 메카로 불리는 걸까. 암베르크 공장은 커다란 병원 수술실 같았다.
4 line :
    not found : [더미에서바늘을] 안내자가 “건초더미에서 바늘을 찾는 게 더 쉬울 것”이라고 말할 정도로 먼지 한 점 없이 깨끗했다.
    found : [ 말할] 안내자가 “건초더미

In [25]:
# 예측한 띄어쓰기 결과로 재생성한 기사
# 문장의 마지막 단어와 다음 line의 첫 단어의 연결 여부를 확인하여 원문에서 그 단어만 교체하였다.
text = ''.join(lines)

idx = 0
for a in answer :
  idx = text.find('\n')
  if a == 1 :
    text = text[:idx+3].replace('\n', ' ') + text[idx+3:]
  else :
    text = text[:idx+3].replace('\n', '') + text[idx+3:]
text

'독일 뮌헨에서 북쪽으로 200㎞ 떨어진 암베르크는 인구 4만 4000명의 작은 도시다. 하지만 앙겔라 메르켈 독일 총리는 물론 전 세계 기업인과 학자들이 4차 산업혁명을 공부하기 위해 찾는 곳이다. 세계적인 전기전자 기업 지멘스가 자랑하는 스마트 공장이 이곳에 있기 때문이다. 이 공장은 어떤 비밀이 있기에 4차 산업혁명의 메카로 불리는 걸까. 암베르크 공장은 커다란 병원 수술실 같았다. 안내자가 “건초 더미에서 바늘을 찾는 게 더 쉬울 것”이라고 말할 정도로 먼지 한점 없이 깨끗했다. 축구장 1.5배인 1만㎡의 널찍한 공간에서 컨베이어벨트는 쉴 새 없이 돌아갔지만 ‘사람’은 거의 보이지 않았다. 의사 수술복과 비슷한 파란색 유니폼을 입은 직원들이 드문드문 눈에 띄었는데, 모니터만 들여다보고 있을 뿐 뭔가를 만들거나 조립하지는 않았다. 암베르크 공장에선 전체 공정의 75%가 인간의 손이 필요 없는 자동화로 진행된다. 이 공장이 만드는 건 ‘시마틱 프로그램 가능 논리 제어 장치’(PLCs)로 불리는 일종의 칩이다. 기계나 로봇을 조종하고 움직이는 ‘두뇌’라고 생각하면 된다. 암베르크·뮌헨(독일) 임주형기자 hermes@seoul.co.kr ▶관련기사 8면'